In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:
data = pd.read_excel("../data/fpl_data/full_df.xlsx")

In [3]:
data

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,value,was_home,yellow_cards,GW,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts,season
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,55,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,50,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
2,Aaron Mooy,MID,Brighton,0.0,0,0,0,0,0.0,60,...,50,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,50,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
4,Abdoulaye Doucouré,MID,Everton,1.3,0,0,20,1,44.6,512,...,55,False,0,1,0.00,0.00,0.00,0.00,0,2020/21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93194,Kyle Walker,DEF,Man City,3.7,0,0,16,0,36.5,369,...,55,False,0,23,0.12,0.14,0.02,1.02,1,2023/24
93195,Jacob Brown,FWD,Luton,-0.7,0,0,1,0,0.5,631,...,49,False,0,23,0.00,0.05,0.05,0.19,0,2023/24
93196,Vicente Guaita,GK,Crystal Palace,0.0,0,0,0,0,0.0,227,...,44,False,0,23,0.00,0.00,0.00,0.00,0,2023/24
93197,Braian Ojeda Rodríguez,MID,Nott'm Forest,0.0,0,0,0,0,0.0,459,...,45,False,0,23,0.00,0.00,0.00,0.00,0,2023/24


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93199 entries, 0 to 93198
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        93199 non-null  object 
 1   position                    93199 non-null  object 
 2   team                        93199 non-null  object 
 3   xP                          93199 non-null  float64
 4   assists                     93199 non-null  int64  
 5   bonus                       93199 non-null  int64  
 6   bps                         93199 non-null  int64  
 7   clean_sheets                93199 non-null  int64  
 8   creativity                  93199 non-null  float64
 9   element                     93199 non-null  int64  
 10  fixture                     93199 non-null  int64  
 11  goals_conceded              93199 non-null  int64  
 12  goals_scored                93199 non-null  int64  
 13  ict_index                   931

In [5]:
data[(data["season"] == "2023/24") & (data["name"] == "Mohamed Salah")][["team", "kickoff_time", "opponent_team", "expected_goal_involvements", "expected_goals", "expected_goals_conceded", "expected_assists"]].tail(14)

,team,kickoff_time,opponent_team,expected_goal_involvements,expected_goals,expected_goals_conceded,expected_assists
83263,Liverpool,2023-10-29T14:00:00Z,16,0.71,0.58,0.45,0.13
83996,Liverpool,2023-11-05T16:30:00Z,12,0.68,0.59,0.81,0.09
84738,Liverpool,2023-11-12T14:00:00Z,4,1.15,0.94,1.57,0.21
85485,Liverpool,2023-11-25T12:30:00Z,13,0.21,0.00,1.35,0.21
86238,Liverpool,2023-12-03T14:00:00Z,10,1.03,0.77,1.22,0.26
86993,Liverpool,2023-12-06T19:30:00Z,17,0.44,0.36,0.25,0.08
87751,Liverpool,2023-12-09T12:30:00Z,8,0.51,0.33,1.74,0.18
88452,Liverpool,2023-12-17T16:30:00Z,14,0.75,0.42,0.63,0.33
89146,Liverpool,2023-12-23T17:30:00Z,1,0.35,0.22,0.80,0.13
89907,Liverpool,2023-12-26T17:30:00Z,6,0.42,0.27,0.76,0.15


In [6]:
prem_list = sorted(data[data["season"]=="2023/24"]["team"].unique())

In [7]:
prem_list.remove("Liverpool")

In [8]:
prem_list

['Arsenal',
 'Aston Villa',
 'Bournemouth',
 'Brentford',
 'Brighton',
 'Burnley',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Fulham',
 'Luton',
 'Man City',
 'Man Utd',
 'Newcastle',
 "Nott'm Forest",
 'Sheffield Utd',
 'Spurs',
 'West Ham',
 'Wolves']

In [9]:
data = data[data["position"] != "GKP"].reset_index(drop=True)
data_use = data[data["season"].isin(["2021/22","2022/23", "2023/24"])].reset_index(drop=True)

In [10]:
data_use.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68733 entries, 0 to 68732
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        68733 non-null  object 
 1   position                    68733 non-null  object 
 2   team                        68733 non-null  object 
 3   xP                          68733 non-null  float64
 4   assists                     68733 non-null  int64  
 5   bonus                       68733 non-null  int64  
 6   bps                         68733 non-null  int64  
 7   clean_sheets                68733 non-null  int64  
 8   creativity                  68733 non-null  float64
 9   element                     68733 non-null  int64  
 10  fixture                     68733 non-null  int64  
 11  goals_conceded              68733 non-null  int64  
 12  goals_scored                68733 non-null  int64  
 13  ict_index                   687

In [11]:
#data_use = data_use[["name", "position", "team", "opponent_team", "minutes", "goals_scored", "goals_conceded", "clean_sheets", "expected_goal_involvements", "expected_goals", "expected_goals_conceded", "expected_assists", "season", "own_goals", "GW", "was_home", "total_points", "value", "starts"]]

In [12]:
#data_use

In [13]:
df2022_23 = data_use[data_use["season"] ==  "2022/23"]

In [14]:
df2022_23

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,value,was_home,yellow_cards,GW,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts,season
25346,Nathan Redmond,MID,Southampton,1.5,0,0,3,0,0.0,403,...,55,False,0,1,0.00,0.00,0.0,0.0,0,2022/23
25347,Junior Stanislas,MID,Bournemouth,1.1,0,0,3,0,0.0,58,...,50,True,0,1,0.00,0.00,0.0,0.0,0,2022/23
25348,Armando Broja,FWD,Chelsea,2.0,0,0,3,0,0.3,150,...,55,False,0,1,0.00,0.00,0.0,0.0,0,2022/23
25349,Fabian Schär,DEF,Newcastle,2.4,0,3,43,1,14.6,366,...,45,True,0,1,0.00,0.00,0.0,0.0,0,2022/23
25350,Jonny Evans,DEF,Leicester,1.9,0,0,15,0,1.3,249,...,45,True,0,1,0.00,0.00,0.0,0.0,0,2022/23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51846,Oliver Skipp,MID,Spurs,2.0,0,0,16,0,0.0,441,...,43,False,0,38,0.01,0.01,0.0,1.5,1,2022/23
51847,Ryan Sessegnon,DEF,Spurs,0.0,0,0,0,0,0.0,436,...,44,False,0,38,0.00,0.00,0.0,0.0,0,2022/23
51848,Ashley Young,DEF,Aston Villa,1.0,0,0,0,0,0.0,538,...,43,True,0,38,0.00,0.00,0.0,0.0,0,2022/23
51849,Jeremy Sarmiento Morante,MID,Brighton,0.0,0,0,0,0,0.0,119,...,45,False,0,38,0.00,0.00,0.0,0.0,0,2022/23


In [15]:
pl2021_22 = pd.read_excel("../data/premier_league_fixtures/pl2021_22.xlsx")
pl2021_22["season"] = "2021/22"
pl2022_23 = pd.read_excel("../data/premier_league_fixtures/pl2022_23.xlsx")
pl2022_23["season"] = "2022/23"
pl2023_24 = pd.read_excel("../data/premier_league_fixtures/pl2023_24.xlsx")
pl2023_24["season"] = "2023/24"

In [16]:
pl2022_23

,GW,Kick Off,Match Time(US EST),Match Time(EET),Match Time(CET),Match Time(WET),Match Time(MSK),Home Team,Home Goal,Away Goal,Away Team,Likely Home Goal,Likely Away Goal,Projected Home Goal,Projected Away Goal,Win %,Draw %,Loss %,season
0,1,2022-08-05,2022-08-05 15:00:00,2022-08-05 20:00:00,2022-08-05 21:00:00,2022-08-05 22:00:00,2022-08-05 22:00:00,Crystal Palace,0,2,Arsenal,1,2,1.090100,1.771,0.232798,0.233874,0.530837,2022/23
1,1,2022-08-06,2022-08-06 07:30:00,2022-08-06 12:30:00,2022-08-06 13:30:00,2022-08-06 14:30:00,2022-08-06 14:30:00,Fulham,2,2,Liverpool,1,2,1.167100,2.161,0.197477,0.202107,0.593401,2022/23
2,1,2022-08-06,2022-08-06 10:00:00,2022-08-06 15:00:00,2022-08-06 16:00:00,2022-08-06 17:00:00,2022-08-06 17:00:00,Bournemouth,2,0,Aston Villa,1,2,1.024100,1.701,0.227374,0.240178,0.530470,2022/23
3,1,2022-08-06,2022-08-06 10:00:00,2022-08-06 15:00:00,2022-08-06 16:00:00,2022-08-06 17:00:00,2022-08-06 17:00:00,Leeds Utd,2,1,Wolves,1,1,1.420100,1.381,0.381842,0.252693,0.364214,2022/23
4,1,2022-08-06,2022-08-06 10:00:00,2022-08-06 15:00:00,2022-08-06 16:00:00,2022-08-06 17:00:00,2022-08-06 17:00:00,Newcastle,2,0,Nottingham Forest,2,0,2.124100,0.641,0.711837,0.182702,0.099234,2022/23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38,2023-05-28,2023-05-28 11:30:00,2023-05-28 16:30:00,2023-05-28 17:30:00,2023-05-28 18:30:00,2023-05-28 18:30:00,Everton,1,0,Bournemouth,2,2,1.519100,1.391,0.404191,0.246703,0.347512,2022/23
376,38,2023-05-28,2023-05-28 11:30:00,2023-05-28 16:30:00,2023-05-28 17:30:00,2023-05-28 18:30:00,2023-05-28 18:30:00,Leeds Utd,1,4,Spurs,1,2,1.420100,1.911,0.286134,0.222222,0.487416,2022/23
377,38,2023-05-28,2023-05-28 11:30:00,2023-05-28 16:30:00,2023-05-28 17:30:00,2023-05-28 18:30:00,2023-05-28 18:30:00,Leicester,2,1,West Ham,1,1,1.132371,1.401,0.303855,0.265031,0.430313,2022/23
378,38,2023-05-28,2023-05-28 11:30:00,2023-05-28 16:30:00,2023-05-28 17:30:00,2023-05-28 18:30:00,2023-05-28 18:30:00,Man Utd,2,1,Fulham,1,1,0.853757,1.181,0.265468,0.299696,0.434577,2022/23


In [17]:
pl2023_24_new = pl2023_24.dropna(subset=["Home Goal", "Away Goal"]).reset_index(drop=True)

In [18]:
pl_all = pd.concat([pl2021_22, pl2022_23, pl2023_24_new], axis = 0).reset_index(drop=True)

In [19]:
def combine_fpl_pl(year):
    pl = pl_all[pl_all["season"] == year].reset_index(drop=True)
    pl = pl[["GW", "Kick Off", "Home Team", "Home Goal", "Away Goal", "Away Team", "Win %", "Draw %", "Loss %"]]
    for i in range(len(pl)):
        if pl.loc[i, "Home Goal"] > pl.loc[i, "Away Goal"]:
            pl.loc[i, "Status"] = "Home Win"
        elif pl.loc[i, "Home Goal"] < pl.loc[i, "Away Goal"]:
            pl.loc[i, "Status"] = "Away Win"
        else:
            pl.loc[i, "Status"] = "Draw"
    pl["Home Team"] = pl["Home Team"].replace("Leeds Utd", "Leeds")
    pl["Away Team"] = pl["Away Team"].replace("Leeds Utd", "Leeds")
    df = data_use[data_use["season"] ==  year]
    
    home_games = df[df["was_home"] == True].reset_index(drop=True)
    total_home = home_games.merge(pl, how = "left", left_on = ["GW", "team"],right_on = ["GW", "Home Team"])
    
    away_games = df[df["was_home"] == False].reset_index(drop=True)
    total_away = away_games.merge(pl, how = "left", left_on = ["GW", "team"],right_on = ["GW", "Away Team"])
    
    tot_rec = pd.concat([total_home, total_away]).reset_index(drop=True)
    for i in range(len(tot_rec)):
        if tot_rec.loc[i, "team"] == tot_rec.loc[i, "Home Team"]:
            tot_rec.loc[i, "opponent"] = tot_rec.loc[i, "Away Team"]
        else:
            tot_rec.loc[i, "opponent"] = tot_rec.loc[i, "Home Team"]
        
    tot_rec = tot_rec.dropna(subset = "Kick Off").reset_index(drop=True)
    tot_rec.index = tot_rec["Kick Off"]
    return tot_rec.sort_index()

In [20]:
df_2021_22 = combine_fpl_pl("2021/22")
df_2022_23 = combine_fpl_pl("2022/23")
df_2023_24 = combine_fpl_pl("2023/24")

In [21]:
tot_df = pd.concat([df_2021_22, df_2022_23, df_2023_24])

In [22]:
tot_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 64859 entries, 2021-08-13 to 2024-02-05
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        64859 non-null  object 
 1   position                    64859 non-null  object 
 2   team                        64859 non-null  object 
 3   xP                          64859 non-null  float64
 4   assists                     64859 non-null  int64  
 5   bonus                       64859 non-null  int64  
 6   bps                         64859 non-null  int64  
 7   clean_sheets                64859 non-null  int64  
 8   creativity                  64859 non-null  float64
 9   element                     64859 non-null  int64  
 10  fixture                     64859 non-null  int64  
 11  goals_conceded              64859 non-null  int64  
 12  goals_scored                64859 non-null  int64  
 13  ict_index     

In [23]:
tot_df.to_excel("../data/fpl_data/fpl_data_pl.xlsx", index=False)